In [ ]:
# Add description here
#

In [ ]:
# Uncomment the next two lines to enable auto reloading for imported modules
# %load_ext autoreload
# %autoreload 2
# For more info, see:
# https://docs.ploomber.io/en/latest/user-guide/faq_index.html#auto-reloading-code-in-jupyter

In [ ]:
# If this task has dependencies, declare them in the YAML spec and leave this
# as None
upstream = None

# This is a placeholder, leave it as None
product = None

In [3]:
# Parameters
swow_en_r100 = (
    "D:\\unibas\\new_embedding_risk_perception\\reanalysis\\data\\raw\\SWOW-EN.R100.csv"
)
product = {
    "nb": "D:\\unibas\\new_embedding_risk_perception\\risk_perception_bk\\pipeline\\reanalysis\\products\\process_swow.ipynb",
    "data": "D:\\unibas\\new_embedding_risk_perception\\risk_perception_bk\\pipeline\\reanalysis\\products\\swow_svdmtx.csv",
}


In [1]:
import pandas as pd 
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [21]:
df_swow = pd.read_csv(swow_en_r100, index_col=0)

In [22]:
df_swow.head()

,id,participantID,age,gender,nativeLanguage,country,education,created_at,cue,R1,R2,R3
1,29,3,33,Fe,United States,Australia,NaN,2011-08-12 02:19:38,although,nevertheless,yet,but
2,30,3,33,Fe,United States,Australia,NaN,2011-08-12 02:19:38,deal,no,cards,shake
3,31,3,33,Fe,United States,Australia,NaN,2011-08-12 02:19:38,music,notes,band,rhythm
4,32,3,33,Fe,United States,Australia,NaN,2011-08-12 02:19:38,inform,tell,rat on,NaN
5,33,3,33,Fe,United States,Australia,NaN,2011-08-12 02:19:38,way,path,via,method


In [23]:
df_swow = df_swow.melt(id_vars=['cue'], value_vars=['R1','R2','R3'], value_name='response').drop('variable',axis=1)

In [24]:
df_swow.head()

,cue,response
0,although,nevertheless
1,deal,no
2,music,notes
3,inform,tell
4,way,path


In [54]:
# remove rows if response counts per cue is less than 5 in df_swow
ts_bigger_response = (df_swow.response.value_counts() >= 5)


In [83]:
df_swow2 = df_swow[df_swow.response.isin(ts_bigger_response.index[ts_bigger_response])].reset_index(drop=True)

In [84]:
df_swow2['counts'] = 1
df_swow2 = df_swow2.groupby(['cue', 'response']).count()['counts'].reset_index().sort_values('counts', ascending=False)

In [85]:
df_swow2.head()

,cue,response,counts
5777,Cheddar,cheese,97
739725,mozzarella,cheese,95
164226,brewery,beer,94
732690,moo,cow,94
234526,cob,corn,94


In [95]:
df_swow3 = (df_swow2.pivot(index='cue', columns='response').fillna(0).astype(int))

In [105]:
# calculate PMI

total_counts = df_swow3.values.sum()

p_x = df_swow3.sum(axis=1) / total_counts
p_y = df_swow3.sum(axis=0) / total_counts
p_x_y = df_swow3 / total_counts

p_x_reshaped = p_x.values.reshape(-1, 1)
p_y_reshaped = p_y.values.reshape(1, -1)

pmi = np.log2(p_x_y / (p_x_reshaped * p_y_reshaped))


c:\Users\by003457\Envs\cbert\lib\site-packages\pandas\core\internals\blocks.py:329: RuntimeWarning: divide by zero encountered in log2
  result = func(self.values, **kwargs)


In [110]:
# convert inf to 0 in pmi
pmi.replace([np.inf, -np.inf], 0, inplace=True)

In [111]:
# ARPACK is an exact solver, ensuring reproducibility
svd = TruncatedSVD(n_components=300, algorithm='arpack')
swow_ppmi_svd = pd.DataFrame(svd.fit_transform(pmi), index=pmi.index.values)
swow_ppmi_svd.shape

(12281, 300)

In [112]:
swow_ppmi_svd

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
Abel,3.746972,0.431669,0.206850,1.395547,-1.860150,0.929966,0.496726,2.843006,1.173654,2.642163,...,0.440834,0.632963,-1.849690,-0.303722,0.411596,-0.212488,-0.185165,0.234732,0.054954,-0.907134
Aboriginal,4.982827,-2.047308,-0.448597,0.006977,-4.110317,1.469082,-1.728615,4.250387,-0.126497,1.333159,...,0.193480,0.918783,-0.961476,-0.216822,0.630551,1.825704,2.797209,-0.644956,0.527600,0.800986
Adam,3.862947,-0.294543,0.199051,2.262865,-1.869041,0.923075,-0.506730,1.836875,0.380007,1.316648,...,1.109901,1.412511,-1.081132,0.009890,0.167784,0.249104,-0.692299,0.237080,0.054690,-0.896367
Advil,2.870456,-1.909342,1.221090,-1.314112,2.501709,-0.806201,0.576651,-1.392784,2.817033,3.760644,...,-1.314678,2.029832,2.026707,-1.863659,-0.533610,-1.534210,0.417730,-0.116602,2.753456,2.005348
Africa,6.808350,-4.214014,-0.670601,-1.088998,-5.247525,1.526843,0.818505,5.629591,0.872141,1.437030,...,-1.858689,-0.854001,4.904829,0.347888,1.288810,-1.587416,-0.264810,-0.577208,-0.247893,-0.561930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zombies,6.630980,-2.584721,4.974413,-1.684895,-0.550795,-2.357493,-3.659213,2.899247,1.436586,1.816519,...,-0.254743,-1.093108,-0.725499,0.877449,0.638779,-0.052266,-1.057700,-0.902933,-1.611977,0.585158
zone,7.132692,0.143281,-2.540941,-4.978508,-2.385324,0.186406,1.786306,2.422127,0.142646,-0.729185,...,0.104056,-0.652240,0.809094,0.726932,-1.036134,-0.950622,-0.119539,-0.201890,0.026869,1.876200
zoo,5.647315,-3.047856,-0.118402,-1.217367,-2.639905,-0.693628,-0.164375,3.002917,-0.721350,0.171067,...,-2.502941,0.222481,0.792764,-0.636359,1.526586,-1.047842,-1.006260,0.747071,0.446998,-0.658329
zoom,5.937176,-0.552861,-3.029134,-4.382398,-2.004359,-3.932158,-1.763769,-0.632503,0.576503,-3.979052,...,-0.300152,-0.638370,-2.253392,0.014123,-0.511410,-0.008481,0.123942,-1.463854,1.950484,1.486973


In [113]:
swow_ppmi_svd.to_csv(product['data'])